# Laboratorio 2 - Perceptrón para clasificar lirios - Práctica 1

Grupo B07

- Álvaro Ramos Morales

- Álvaro Delgado Gallego

- Fernando Ramírez Fernández

- Juan Esteban Bernal Santos

In [ ]:
# Se importan las librerías necesarias para el desarrollo de la práctica
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, make_scorer
from tabulate import tabulate


## Estudio del dataset Iris

In [ ]:
# Cargamos el conjunto de datos de iris
iris = load_iris()
X, y = iris.data, iris.target

# Ver las primeras 5 filas de datos
print("\nPrimeras 5 filas de datos:")
print(iris.data[:5])

# Ver las etiquetas de las clases
print("\nEtiquetas de las clases:")
print(iris.target_names)

# Ver la descripción del conjunto de datos
print("\nDescripción del conjunto de datos:")
print(iris.DESCR)





In [ ]:
class Perceptron:
    def __init__(self, learning_rate=0.1, threshold=0.1, n_iter=1000):
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.n_iter = n_iter
        self.weights = None
        self.bias = None
        self.history = []
    
    def activate(self, x):
        return np.where(x >= self.threshold, 1, 0)
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        reached_max_accuracy = False

        for epoch in range(self.n_iter):
            accuracy = accuracy_score(y, self.predict(X))
            epoch_history = {'Epoch': epoch + 1, 'Updates': [], 'Accuracy': accuracy}
            if accuracy == 1 and not reached_max_accuracy:
                reached_max_accuracy = True
            elif reached_max_accuracy and accuracy == 1:
                self.history.append(epoch_history)
                break  # Finaliza después de una época extra después de alcanzar accuracy de 1

            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activate(linear_output)
                update = self.learning_rate * (y[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update

                # Ahora guarda todas las instancias procesadas, independientemente del valor de update
                epoch_history['Updates'].append({
                    'Inputs': x_i.tolist(),
                    'Desired output': y[idx],
                    'Actual output': y_predicted,
                    'Error': y[idx] - y_predicted,
                    'Initial weights': self.weights.tolist(),
                    'Final weights': (self.weights + update * x_i).tolist()
                })
            self.history.append(epoch_history)
            
                
    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activate(linear_output)
        return y_predicted

In [ ]:
# Función para graficar puntos de distinto color para cada tipo de lirio
def plot_iris_data(X, y):
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, palette='Set1', legend='full')
    plt.xlabel('Longitud del sépalo')
    plt.ylabel('Anchura del sépalo')
    plt.title('Distribución de los tipos de lirio')
    plt.grid(True)
    plt.show()

In [ ]:
#Funcion dibujar hiperplano
def plot_hyperplane(X, y, weights, bias):
    m = -weights[0] / weights[1]
    b = -bias / weights[1]
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, palette='Set1', legend='full')
    plt.xlabel('Longitud del petalo')
    plt.ylabel('Anchura del petalo')
    plt.title('Distribución de los tipos de lirio')
    plt.grid(True)
    
    x_hyperplane = np.linspace(0, 8, 10)
    y_hyperplane = (-bias - weights[0] * x_hyperplane ) / weights[1]
    #mostrar ecuacion del hiperplano en el grafico de forma explicita
    # Transformación a forma explícita

    plt.plot(x_hyperplane, y_hyperplane, color='black')
    plt.figtext(0.5, 0.01, f'y = {m:.2f}x + {b:.2f}', fontsize=12, ha='center')
    plt.show()


## Cuestión 1

In [ ]:
feature_names = iris.feature_names

# Creamos un DataFrame para facilitar la visualización
iris_df = pd.DataFrame(X, columns=feature_names)
iris_df['Tipo de flor'] = y
species_names = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
iris_df['Tipo de flor'] = iris_df['Tipo de flor'].map(species_names)

# Generamos gráficas de dispersión para cada par de atributos
g = sns.pairplot(iris_df, hue='Tipo de flor', palette='Set1', diag_kind='None')

plt.show()


Debido a la relación observada entre cada par de variables dentro del dataset, se determina que el par más adecuado para la clasificación es el de "petal length" (longitud del pétalo) con "petal width" (anchura del pétalo). Esta selección se basa en varias consideraciones: la combinación de "petal length" y "petal width" muestra una distinción más notoria entre las tres clases de flores (setosa, versicolor y virginica) en comparación con otros pares de variables, lo que significa que estas dos características tienen la capacidad de separar eficazmente las diferentes clases de flores en el espacio de características. 

Además, al observar los valores de las características dentro de cada clase, se nota que "petal length" y "petal width" exhiben patrones de comportamiento más similares dentro de una misma clase, lo que los hace más consistentes y distintivos para cada tipo de flor

In [ ]:

#cogemos sepal length y sepal width
X = iris.data[:,(0,1)]



#X = iris.data[:,(2,3)]  # Tomamos los dos atributos con mayor relevancia para la clasificación
# Convertir las etiquetas de las clases a 0 y 1
y = (iris.target != 0) * 1


#Cojemos solo las plantas de tipo setosa y versicolor
X = X[y != 2]
y = y[y != 2]

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Definir los hiperparámetros para el grid search
learning_rates = [0.01, 0.1, 0.5]
thresholds = [0.01, 0.2, 0.75]

results_list = []

# Crear gridsearch y recoger historial
#El for muestre 1 valor mas y no pare cuando accuracy es 1

for lr in learning_rates:
    for th in thresholds:
        perceptron = Perceptron(learning_rate=lr, threshold=th)
        perceptron.fit(X_train, y_train)
        y_pred = perceptron.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        # Agregar los resultados a la lista
        results_list.append({
            'Learning Rate': lr,
            'Threshold': th,
            'Accuracy': accuracy,
            'History': perceptron.history
        })

# Convertir la lista de resultados en un DataFrame
results = pd.DataFrame(results_list)



# Entrenamos el perceptrón
perceptron = Perceptron(learning_rate=0.01, threshold=0.01) 
perceptron.fit(X_train, y_train)

#comprobar si tiene overfitting
y_pred_train = perceptron.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
y_pred_test = perceptron.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)

print(f'Accuracy en entrenamiento: {accuracy_train:.2f}')
print(f'Accuracy en prueba: {accuracy_test:.2f}')

# Mostrar los resultados en una tabla con import tabulate y solucionar el problema de que no se muestra toda la tabla
for idx, row in results.iterrows():
    print(f'\nResultados para Learning Rate: {row["Learning Rate"]} y Threshold: {row["Threshold"]}')
    print(f'Accuracy: {row["Accuracy"]:.2f}')
    print('Historial de actualización de pesos:')
    for epoch in row['History']:
        print(f'\nÉpoca: {epoch["Epoch"]}, Accuracy: {epoch["Accuracy"]:.2f}')
        for update in epoch['Updates']:
            print(f'Entradas: {update["Inputs"]}, Salida deseada: {update["Desired output"]}, Salida actual: {update["Actual output"]}, Error: {update["Error"]}, Pesos iniciales: {update["Initial weights"]}, Pesos finales: {update["Final weights"]}')



## Cuestión 2

In [ ]:
# Graficamos la distribución de los tipos de lirio
plot_iris_data(X, y)

In [ ]:
print("Pesos del perceptrón:", perceptron.weights)
print("Umbral del perceptrón:", perceptron.bias)

In [ ]:
# Graficamos el hiperplano resultante
plot_hyperplane(X, y, perceptron.weights, perceptron.bias)

## Cuestión 3

In [ ]:
# Preparar datos para el primer Perceptrón (3 atributos)
X_train_3 = X_train[:, :3]
X_test_3 = X_test[:, :3]

# Preparar datos para el segundo Perceptrón (4 atributos)
X_train_4 = X_train  # Usar los 4 atributos
X_test_4 = X_test

# Inicializar y entrenar el primer Perceptrón (3 atributos)
perceptron_3 = Perceptron(learning_rate=0.1, threshold=0.1)
perceptron_3.fit(X_train_3, y_train)
y_pred_3 = perceptron_3.predict(X_test_3)
accuracy_3 = accuracy_score(y_test, y_pred_3)

# Inicializar y entrenar el segundo Perceptrón (4 atributos)
perceptron_4 = Perceptron(learning_rate=0.1, threshold=0.1)
perceptron_4.fit(X_train_4, y_train)
y_pred_4 = perceptron_4.predict(X_test_4)
accuracy_4 = accuracy_score(y_test, y_pred_4)

print(f"Accuracy con 3 atributos: {accuracy_3}")
print(f"Accuracy con 4 atributos: {accuracy_4}")

#Comparamos los resultados con los obtenidos en el apartado anterior
print("Pesos del perceptrón con 3 atributos:", perceptron_3.weights)
print("Umbral del perceptrón con 3 atributos:", perceptron_3.bias)
print("Pesos del perceptrón con 4 atributos:", perceptron_4.weights)
print("Umbral del perceptrón con 4 atributos:", perceptron_4.bias)


DUDAS¿?
-Es normal que saque accuracy 1
-Hay que estudiar el n iteraciones
-Mismos resultado con varios atributos 